# Examples from the discrete world

The purpose of this notebook is to test our general methodology in a very basic setting. In this sense, 
1. we will assume a model in a discrete setting (e.g., a basic binomial model); 
2. we will generate the needed data from that model and "assume" we don't know from which model this data comes from;
3. and try to solve the problem for this data.

## 1. Create the model

In [9]:
from robust_pricing.path_generators import BinomialTree, Uniform, Gaussian, GaussianMartingale, UniformMartingale
from helper import *

In [361]:
granularity=100
path_length=2
# up = 1.030454533953517
volatility = 0.15
up = np.exp(volatility * np.sqrt(1 / granularity))
model = BinomialTree(
    path_length=path_length,
    mean=100,
    up_factor=up,
    down_factor=1 / up,
    granularity=100,
    observed_times=[i * (granularity // path_length) for i in range(1, path_length + 1)],
)

In [362]:
model.up_factor

1.015113064615719

In [363]:
data = model(10000).numpy()

In [452]:
hist_2d(data, 0, 1)

## 2. Create the inputs

In [453]:
%reload_ext autoreload
%autoreload 2
from robust_pricing.deePricing import *


## 2.1 The option portfolios

Based on this model, we need to create the base set of options $\mathcal{B}_t$ from which our (optimization) model will try to learn the model (probability measure) from which they come. For this task, we use the fact that we can sample from this model to price them using Monte-Carlo.

In [454]:
strike_prices = list(range(70, 130, 1))
pricing_data = model(10000)

option_portfolios = []

for t in range(path_length):
    calls = []
    puts = []
    for k in strike_prices:
        c = Call(strike=k, price=0)
        c.price = float(torch.mean(c(pricing_data[:, t])))
        p = Put(strike=k, price=0)
        p.price = float(torch.mean(p(pricing_data[:, t])))
        calls.append(c)
        puts.append(p)

    option_portfolios.append(
        OneMaturityOptionPortfolio(calls=calls, puts=puts)
    )
option_portfolio = DiagonalOptionPortfolio(option_portfolios)

## 2.2 The exotic option

In this case we will use a simple call option:

In [455]:
K = 100
def f(x):
    if isinstance(x, torch.Tensor):
        return torch.relu(x.select(1,-1) - K)
    else:
        return max(x[-1] - K, 0)

# def f(x):
#     if isinstance(x, torch.Tensor):
#         return torch.relu(x.select(1,-1) - torch.mean(x, 1))
#     else:
#         return max(x[-1] - sum(x)/len(x), 0)

Since we know the "right" model, we can use it to price the option using Monte-Carlo.

In [456]:
torch.mean(f(pricing_data))

tensor(5.7054)

In [457]:
option_portfolio.option_portfolios[-1].calls

[Call(strike=70, price=29.753503799438477),
 Call(strike=71, price=28.765403747558594),
 Call(strike=72, price=27.778118133544922),
 Call(strike=73, price=26.797618865966797),
 Call(strike=74, price=25.817119598388672),
 Call(strike=75, price=24.849010467529297),
 Call(strike=76, price=23.88201141357422),
 Call(strike=77, price=22.927583694458008),
 Call(strike=78, price=21.979583740234375),
 Call(strike=79, price=21.039031982421875),
 Call(strike=80, price=20.113130569458008),
 Call(strike=81, price=19.18722915649414),
 Call(strike=82, price=18.295024871826172),
 Call(strike=83, price=17.404924392700195),
 Call(strike=84, price=16.53609275817871),
 Call(strike=85, price=15.69099235534668),
 Call(strike=86, price=14.845893859863281),
 Call(strike=87, price=14.047050476074219),
 Call(strike=88, price=13.251749992370605),
 Call(strike=89, price=12.475825309753418),
 Call(strike=90, price=11.743525505065918),
 Call(strike=91, price=11.011225700378418),
 Call(strike=92, price=10.3215618133

## 2.3 We need to choose a model $\theta$ to sample from

In [458]:
gm = Uniform(path_length, mean=model.mean, variance=80 ** 2)
# gm = Gaussian(path_length, mean=model.mean, variance=30 ** 2)
# gm = GaussianMartingale(path_length, mean=model.mean, variance=20 ** 2)
# gm = UniformMartingale(path_length, mean=model.mean, variance=50 ** 2)

# gm = model
data = gm(1000)

In [459]:

hist_2d(data, 0, 1)

## 2.4 Create the hedging strategies

In [460]:
from math import prod
s = 0
for par in h_sup.parameters():
    s += prod(list(par.shape))
s

10641

## 2.5 Optimize

In [ ]:
from copy import deepcopy

number_of_observations=1000
gamma = 1000
no_trading_strategy = False


option_portfolio.reset_weights(zeros=True)

h_sup = HedgingStrategy(
    option_portfolio=deepcopy(option_portfolio),
    target_function=f,
    super_hedge=True,
    trading_kwargs=dict(num_layers=5, width_layers=100),
    no_trading_strategy=no_trading_strategy
)
h_sub = HedgingStrategy(
    option_portfolio=deepcopy(option_portfolio),
    target_function=f,
    super_hedge=False,
    trading_kwargs=dict(num_layers=5, width_layers=100),
    no_trading_strategy=no_trading_strategy    
)


h_sup.penalization_function.gamma = gamma
h_sub.penalization_function.gamma = gamma


h_sup.reset_weights(zeros=True)
h_sup.optimize(
    generator=gm, 
    number_of_observations=number_of_observations, 
    number_of_episodes=None,
)

h_sub.reset_weights(zeros=True)
h_sub.optimize(
    generator=gm, 
    number_of_observations=number_of_observations, 
    number_of_episodes=None,
)

In [ ]:
h_sup.visualize_training_status()

In [ ]:
h_sub.visualize_training_status()

In [ ]:
h_sup.training_status.keys()

## 2.6 "Validate"

Since in this example, we have the actual model from which the option prices come from, we can estimate the replication error against that model.

In [ ]:
print(h_sup.replication_error(model(1000)))
print(h_sub.replication_error(model(1000)))

We can print the upper and lower bound for the option price given the observed prices:

In [ ]:
print(f'Upper bound: {h_sup.value}')
print(f'Actual bound: {torch.mean(f(pricing_data))}')
print(f'Lower bound: {h_sub.value}')

## 3. Let's evaluate the impact of $\gamma$

Recall that the objective function we are minimizing penalizes the sup/sub-hedging constraint with a penalization function $\beta:\mathbb{R}\to\mathbb{R}_+$. In this example we are using 

$$ \beta(x) = \frac{1}{p}(\max\{x, 0 \})^p,$$
with $p=2$. For any penalization function $\beta$, we define and we define $\beta_\gamma:\mathbb{R}\to\mathbb{R}_+$

$$\beta_\gamma(x)=\frac{1}{\gamma}\beta(\gamma x).$$

In [ ]:
low=1 
high=41
step=1 

lower_bound, upper_bound, sup_replication_error, sub_replication_error = run_for_gammas_in(
    generator=model, 
    test_data=model(1000),
    option_portfolio=option_portfolio,
    target_function=f,
    low=low, 
    high=high,
    step=step, 
    number_of_observations=500, 
    number_of_episodes=3000,
    no_trading_strategy=False,
    zeros=True
) 

In [ ]:
import plotly.graph_objects as go
# Create traces
fig = go.Figure()
x = list(range(low, high, step))

fig.add_trace(go.Scatter(
    x=x,
    y=sup_replication_error,
    mode='lines',
    name='sup_replication_error')
)

fig.add_trace(go.Scatter(
    x=x,
    y=sub_replication_error,
    mode='lines',
    name='sub_replication_error')
)


fig.update_layout(
    title="",
    xaxis_title=r"$\gamma$",
    yaxis_title=f"Replication error",
    legend_title="Bounds",

)

fig.show()

In [ ]:
# Create traces
fig = go.Figure()
x = list(range(low, high, step)) 

fig.add_trace(go.Scatter(
    x=x,
    y=lower_bound,
    mode='lines',
    name='lower_bound')
)

fig.add_trace(go.Scatter(
    x=x,
    y=upper_bound,
    mode='lines',
    name='upper_bound')
)

fig.update_layout(
    title="",
    xaxis_title=r"$\gamma$",
    yaxis_title=f"Solution",
    legend_title="Bounds",

)

fig.show()

## References
[1] Guo, G., & Obłój, J. (2019). Computational methods for martingale optimal transport problems. Annals of Applied Probability, 29(6), 3311–3347. https://doi.org/10.1214/19-AAP1481